<a href="https://colab.research.google.com/github/afonso-tiago/thesis-notebooks/blob/main/tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intoduction 

This is one of the complementary notebooks to the bachelor thesis titled "*Comparing Performance of Different Goal Functionals in Solving PDEs Using Neural Networks*". It contains the complete code to perform every test described in the thesis and is written in such a way to allow easy construction of own new tests.

> A good way to navigate Google Colaboratory notebooks is by using the built-in table of contents; the first item in the menu bar on the left side of the screen.

> Helpful keyboard shortcuts are: 
* <kbd>Shift</kbd> + <kbd>Enter</kbd>: executes a cell and jump to the next one
* <kbd>Ctrl</kbd> + <kbd>Enter</kbd>: executes a cell but stays in that cell



# Preparation

We can easily enable hardware acceleration by navigating to `Edit`
and clicking on the `Notebook Settings`. This should open a popup where we can select a hardware accelerator. Selecting None or GPU will work immediately without adjustments to the code. If we want to use TPUs we need make some smaller changes as TPUs are usually used for distributed computing inside a cluster.


The following cell tests which GPUs are connected to our notebook

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

A list of all necessary imports

In [ ]:
import math

import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import qmc
# !pip install scipy --upgrade

from IPython.display import clear_output as clear

from datetime import datetime
import time

from collections import defaultdict

from os import listdir
from pathlib import Path
import re

# Load the TensorBoard notebook extension
%load_ext tensorboard
log_base_route = "logs"

If you want to save log files permanently you can connect your google drive by running this cell. **Otherwise just skip this cell.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# adjust the base route of the log files such that they are directly saved to the drive
log_base_route = "/content/drive/MyDrive/PINN_vs_DRM_tests/logs"

# Interior and Boundary Point Generation
In this section we will construct all necessary generators
and create right-hand sides and exact solutions corresponding to different settings of the Poisson problem. The theoretical counterpart to this is **section 2.3** in the thesis. 

## Generators

This cell contains various generators for the interior and boundary of different domains.

*   interior of hypercube randomly
*   interior of hypercube via LHS
*   interior of n-sphere randomly
*   interior of pie shaped domain randomly
*   boundary of hypercube regularly
*   boundary of hypercube randomly
*   boundary of hypercube via LHS
*   boundary of n-sphere randomly
*   boundary of pie shaped domain randomly

A generator always accepts the parameters `d` (dimension), 
some domain parameters (here `a, b` or `R`) and interior or boundary parameters (here `batch_size_inner`, `num_in_one_hyperplane` or `batch_size_boundary`).
By following this standard, new generators can be easily added.

In [ ]:
# # # # # # # # # # # # # 
# interior generator(s) #
# # # # # # # # # # # # # 

def generate_interior_randomly(d, a, b, batch_size_inner):
  # uniformly generate random points inside the hypercube
    # we unstack along axis 0 in order to convert tensor of shape (d, batch_size_inner) 
    # into d lists of tensors/values (the "x1"-values, the "x2"-values, ...)
  X_inner = tf.unstack(tf.random.uniform([d, batch_size_inner], a, b), axis=0) 

  # calc d dimensional "volume" of interior
  V_inner = abs(b-a)**d

  return X_inner, V_inner

def generate_interior_via_LHS(d, a, b, batch_size_inner):
  # generate points using Latin Hypercube sampling
  sample = qmc.LatinHypercube(d=d).random(n=batch_size_inner)
  sample = qmc.scale(sample, d*[a], d*[b])
    # we unstack along axis 1 in order to convert batch_size_inner points into d lists of values (the "x1"-values, the "x2"-values, ...)
  X_inner = tf.unstack(tf.convert_to_tensor(sample, dtype='float32'), axis=1) 

  # calc d dimensional "volume" of interior
  V_inner = abs(b-a)**d

  return X_inner, V_inner

def generate_interior_n_sphere_randomly(d, R, batch_size_inner):
  # This algorithm is due to Marsaglia 1972 and Muller (1959)

  # generate normal deviates and normalize them 
  X, _ = tf.linalg.normalize(tf.random.normal([d, batch_size_inner]), ord=2, axis=0)
  # uniformly generate random radii and rescale them 
  r = R*tf.pow(tf.random.uniform([batch_size_inner]), 1/d)
  # scale the normalized normal deviates by r
  X_inner = tf.unstack(r*X, axis=0)

  # calc d dimensional "volume" of interior
  if d%2 == 0:
    V_inner = math.pi*R**2
    start_dim = 4
  else: 
    V_inner = 2*R
    start_dim = 3
  # we calculate the volume iteratively (instead via the closed form formula) to avoid large factorials and powers
  # this way we can for example calc the volume for d=200 R=1.0 or d=2000 R=10.0
  for k in range(start_dim,d+1,2):
      V_inner *= 2*math.pi*R**2/k

  return X_inner, V_inner

def generate_interior_pie_randomly(d, gamma, R, batch_size_inner):
  # Note that parameter d is passed but intentionally not used

  # uniformly generate random polar coordinates
  phi = tf.random.uniform([batch_size_inner], 0.0, gamma)
  r = tf.sqrt(tf.random.uniform([batch_size_inner], 0.0, 1.0))*R # sqrt is necessary to have the same density near 0 as near R

  # convert polar coordinates into cartesian coordinates
  X_inner = [r*tf.cos(phi), r*tf.sin(phi)]

  # calc 2D "volume" of interior
  V_inner = gamma/2*R**2

  # return generated inner points
  return X_inner, V_inner

# # # # # # # # # # # # #
# boundary generator(s) #
# # # # # # # # # # # # #

def generate_boundary_regularly(d, a, b, num_along_one_axis):
  X_boundary = []

  base = [ [a + (b-a)*i/(num_along_one_axis-1)] for i in range(num_along_one_axis)]
  # this method recursively generates all points of the hypercube [a,b]^d
  # generate(0, False) = [ [a, generate(1, True)], [a+(b-a)*1/n, generate(1, False)], ..., [b, generate(1, True)] ]
  #     .
  #     .
  #     .
  # generate(d-1, True) = base = [ [a], [a+(b-a)*1/n] [a+(b-a)*2/n] ..., [b] ] (where n is num_along_one_axis-1) 
  # the parameter include_all_below informs layers one recursion level below that ALL points need to be generated
  # this happens when the set of points already includes a or b, which means that we are on the boundary
  def generate(depth, include_all_below):
    points = []
    # with each recursive call one dimension is added to the points in the list points
    # because we only need d dimensional points, we need to stop the recursion at depth == d-1
    if depth == d-1:
      # only including the boundary points at the last recusion layer means only including a and b
      points += base if include_all_below else [[a],[b]]
    else:
      # generate all points below (those are always needed for [a, generate()] and [b, generate()])
      all_points_below = generate(depth+1, True)
      # only generate points below with include_all_below = False when needed
      points_below = all_points_below if include_all_below else generate(depth+1, False)

      # [a, generate(depth+1, True)]
      points += [tf.concat([all_points_below, tf.fill([len(all_points_below),1], a)], axis=1)]
      for i in range(1,num_along_one_axis-1): 
          # [a+(b-a)*i/n, generate(depth+1, include_all_below)]
          points += [tf.concat([points_below, tf.fill([len(points_below),1], a+(b-a)*i/(num_along_one_axis-1))], axis=1)]
      # [b, generate(depth+1, True)]
      points += [tf.concat([all_points_below, tf.fill([len(all_points_below),1], b)], axis=1)]
      # concat all seperate lists of points to one list
      points = tf.concat(points, axis=0)
    
    return points
  # convert list of points into d lists of values (the "x1"-values, "x2"-values, ...)
  X_boundary = tf.unstack(generate(0, False), axis=1)

  # calc d-1 dimensional "volume" of boundary
  V_boundary = 2*d*abs(b-a)**(d-1)

  return X_boundary, V_boundary

def generate_boundary_randomly(d, a, b, num_in_one_hyperplane):
  hyperplane_list = []
  for i in range(d):
    for s in (a, b):
        # we unstack along axis 0 in order to convert tensor of shape (d-1, num_in_one_hyperplane) 
        # into d-1 lists of tensors/values (the "x1"-values, the "x2"-values, ...)
      hyperplane = tf.unstack(tf.random.uniform([d-1, num_in_one_hyperplane], a, b), axis=0) 
      hyperplane.insert(i, tf.fill([num_in_one_hyperplane], s))

      hyperplane_list += [hyperplane]
  X_boundary = tf.unstack(tf.concat(hyperplane_list, axis=1), axis=0)

  # calc d-1 dimensional "volume" of boundary
  V_boundary = 2*d*abs(b-a)**(d-1)

  return X_boundary, V_boundary

def generate_boundary_via_LHS(d, a, b, num_in_one_hyperplane):
  hyperplane_list = []
  for i in range(d):
    for s in (a, b):
      # generate points using Latin Hypercube sampling
      sample = qmc.LatinHypercube(d=d-1).random(n=num_in_one_hyperplane)
      sample = qmc.scale(sample, (d-1)*[a], (d-1)*[b])
      
      # we unstack along axis 1 in order to convert num_in_one_hyperplane points into d-1 lists of values (the "x1"-values, the "x2"-values, ...)
      hyperplane = tf.unstack(tf.convert_to_tensor(sample, dtype='float32'), axis=1)
      hyperplane.insert(i, tf.fill([num_in_one_hyperplane], s))

      hyperplane_list += [hyperplane]
  X_boundary = tf.unstack(tf.concat(hyperplane_list, axis=1), axis=0)

  # calc d-1 dimensional "volume" of boundary
  V_boundary = 2*d*abs(b-a)**(d-1)

  return X_boundary, V_boundary

def generate_boundary_n_sphere_randomly(d, R, batch_size_boundary):
  # This algorithm is due to Marsaglia 1972 and Muller (1959)

  # generate normal deviates and normalize them 
  X, _ = tf.linalg.normalize(tf.random.normal([d, batch_size_boundary]), ord=2, axis=0)
  X_boundary = tf.unstack(R*X, axis=0)

  # calc d dimensional "volume" of boundary
  if d%2 == 0:
    V_boundary = 2*math.pi*R
    start_dim = 4
  else: 
    V_boundary = 2
    start_dim = 3
  # we calculate the volume iteratively (instead via the closed form formula) to avoid large factorials and powers
  # this way we can for example calc the volume for d=200 R=1.0 or d=2000 R=10.0
  for k in range(start_dim,d+1,2):
      V_boundary *= 2*math.pi*R**2/(k-2)

  return X_boundary, V_boundary

def generate_boundary_pie_randomly(d, gamma, R, num_boundary_0, num_boundary_1, num_boundary_2):
  # Note that parameter d is passed but intentionally not used

  # generate boundary points for the line where phi=0
  r = tf.random.uniform([num_boundary_0], 0, R)
  X_boundary_0 = [r*math.cos(0), r*math.sin(0)]
  
  # generate boundary points for the arc where phi in [0, gamma]
  phi = tf.random.uniform([num_boundary_1], 0, gamma)
  X_boundary_1 = [R*tf.math.cos(phi), R*tf.math.sin(phi)]

  # generate boundary points for the line where phi=gamma
  r = tf.linspace(0.0, R, num_boundary_2)
  X_boundary_2 = [r*math.cos(gamma), r*math.sin(gamma)]

  # combine all generated boundary points
    # after concatenating all boundary points we get a tensor of shape (d, num_boundary_0 + num_boundary_1 + num_boundary_2) 
    # but because we later need lists of tensors/values (the "x"-values, the "y"-values) we unstack the tensor along axis 0
  X_boundary = tf.unstack(tf.concat([X_boundary_0, X_boundary_1, X_boundary_2], axis=1), axis=0)

  # calc 1D "volume" of boundary
  V_boundary = gamma*R

  # return generated boundary points
  return X_boundary, V_boundary

## Right-Hand Side and Exact Solution

The following functions define the different right-hand sides and boundary conditions. As the boundary condition is always included in the exact solution and we are only interested in examples where we know the exact solution in order to calculate the approximation error, we decided to directly define the exact solution instead of the boundary condition. 

The parameters of the functions are the domain parameters (e.g. `a, b` or `R`) that have to match the domain parameters of the generator used later on during training and `X` containing the points where we want to evaluate the right-hand side or exact solution.

The parameter `X` is an `n` dimensional array, 
where `X[i]` corresponds to a list of the i-th components of all points inside `X`, i.e. `X[i][j]` is the i-th component of the j-th point.

The right-hand sides and exact solutions defined here correspond to the following sections in the thesis:

*   `sum_pow_x_2` $\rightarrow$ **section 3.1** Finite Differences vs Autodiff in High Dimensions
*   `sol_sum_abs_x_i` $\rightarrow$ **section 3.3** The Solution Has no Weak $2^\text{nd}$ Derivative
*   `rhs_Q_Han_F_Lin_page_65` $\rightarrow$ **section 3.4** The $2^\text{nd}$ Derivative of the Solution Explodes

In [ ]:
# DRM paper d=100 example
def rhs_minus_2_d(a, b, X): # start rhs functions with rhs to make better use of auto complete
  return tf.fill([len(X[0])], -2.0*len(X)) 
# the last function defined as rhs will be visualized 
rhs = rhs_minus_2_d 

def sol_sum_pow_x_2(a, b, X):
  return tf.reduce_sum(tf.pow(X, 2), axis=0)
# the last function defined as exact_sol will be visualized
exact_sol = sol_sum_pow_x_2

# parameters defining the domain and boundary
  # this is only needed for visualization
domain_params = [0.0, 1.0] # [a,b]
generate_interior = generate_interior_randomly
generate_boundary = generate_boundary_randomly
interior_params, boundary_params, interior_check_params = [1_000], [100], [10_000]

In [ ]:
# re-entrant corner
def rhs_zero(gamma, R, X):
  return tf.zeros(len(X[0]))
rhs = rhs_zero

def sol_r_sin_phi(gamma, R, X):
  r = tf.sqrt(tf.reduce_sum(tf.pow(X, 2), axis=0))
  phi = tf.math.floormod(tf.atan2(X[1], X[0]), 2*math.pi) # mod 2*pi to change atan2 range of [-pi,pi] to [0, 2*pi]

  return tf.pow(r, math.pi/gamma)*tf.sin(phi*math.pi/gamma)
exact_sol = sol_r_sin_phi

# parameters defining the domain and boundary
  # this is only needed for visualization
domain_params = [1.5*math.pi, 1.0] # [gamma, R]
generate_interior = generate_interior_pie_randomly
generate_boundary = generate_boundary_pie_randomly
interior_params, boundary_params, interior_check_params = [1_000], [10, 100, 10], [10_000]

In [ ]:
# no weak 2nd derivative
def rhs_zero(a, b, X): 
  return tf.fill([len(X[0])], 0.0) 
rhs = rhs_zero 

def sol_sum_abs_x_i(a, b, X): 
  return tf.reduce_sum(tf.abs(X), axis=0)
exact_sol = sol_sum_abs_x_i 

# parameters defining the domain and boundary
  # this is only needed for visualization
domain_params = [-1.0, 1.0] # [a,b]
generate_interior = generate_interior_randomly
generate_boundary = generate_boundary_randomly
interior_params, boundary_params, interior_check_params = [1_000], [100], [10_000]

In [ ]:
# f, g in C but u not in C2
def rhs_Q_Han_F_Lin_page_65(R, X):
  assert len(X) >= 2
  d = len(X)
  r = tf.sqrt(tf.reduce_sum(tf.pow(X, 2), axis=0))
  m_ln_r = -tf.math.log(r)
  return tf.where(r == 0, 0.0, (X[0]**2-X[1]**2)/(2*r**2)*((d+2)/tf.pow(m_ln_r, 0.5) + 1/(2*tf.pow(m_ln_r, 1.5))))
rhs = rhs_Q_Han_F_Lin_page_65

def sol_Q_Han_F_Lin_page_65(R, X):
  assert len(X) >= 2
  r = tf.sqrt(tf.reduce_sum(tf.pow(X, 2), axis=0))
  return tf.where(r == 0, 0.0, (X[0]**2-X[1]**2)*tf.pow(-tf.math.log(r), 0.5))
exact_sol = sol_Q_Han_F_Lin_page_65

# parameters defining the domain and boundary
  # this is only needed for visualization
domain_params = [0.5] # [R]
generate_interior = generate_interior_n_sphere_randomly
generate_boundary = generate_boundary_n_sphere_randomly
interior_params, boundary_params, interior_check_params = [1_000], [1_000], [10_000]

The following are other possible right-hand sides and exact solutions not included in the thesis.

In [ ]:
# no weak 2nd derivative
def rhs_zero(a, b, X): 
  return tf.fill([len(X[0])], 0.0) 
rhs = rhs_zero 

def sol_prod_abs_x_i(a, b, X): 
  return tf.reduce_prod(tf.abs(X), axis=0)
exact_sol = sol_prod_abs_x_i 

# parameters defining the domain and boundary
  # this is only needed for visualization
domain_params = [-1.0, 1.0] # [a,b]
generate_interior = generate_interior_randomly
generate_boundary = generate_boundary_randomly
interior_params, boundary_params, interior_check_params = [1_000], [100], [10_000]

In [ ]:
# unbounded 2nd derivative
def rhs_r_pow_minus_d_half(R, X):
  d = len(X)
  r = tf.sqrt(tf.reduce_sum(tf.pow(X, 2), axis=0))
  return (d/2)*(d/2-2)*tf.minimum(tf.pow(r, -d/2), 1000) # min(..., 1000) avoids infinities
rhs = rhs_r_pow_minus_d_half

def sol_r_pow_2_minus_d_half(R, X): 
  d = len(X)
  r = tf.sqrt(tf.reduce_sum(tf.pow(X, 2), axis=0))
  return tf.minimum(tf.pow(r, 2-d/2), 1000) # min(..., 1000) avoids infinities
exact_sol = sol_r_pow_2_minus_d_half

# parameters defining the domain and boundary
  # this is only needed for visualization
domain_params = [1.0] # [R]
generate_interior = generate_interior_n_sphere_randomly
generate_boundary = generate_boundary_n_sphere_randomly
interior_params, boundary_params, interior_check_params = [1_000], [1_000], [10_000]

## Visualization

In [ ]:
d = 2 # <--- you can adjust the input dimension by changing this value
      #      the visualization of re-entrant corners only works for d=2
# the meaning of the content of interior_params, boundary_params and interior_check_params depends on the generator used

X_inner, _ = generate_interior(d, *domain_params, *interior_params)
f = rhs(*domain_params, X_inner)
X_check, _ = generate_interior(d, *domain_params, *interior_check_params)
u_exact = exact_sol(*domain_params, X_check)
X_boundary, _ = generate_boundary(d, *domain_params, *boundary_params)
g = exact_sol(*domain_params, X_boundary)

if d == 2:  
  # create a figure with three subplots (left, middle, right)
  fig = make_subplots(rows=1, cols=3, 
                      subplot_titles=("points: inner, height: f", "points: check, height: u_exact", "points: boundary, height: g"), 
                      specs=[[{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}]])
  fig.update_layout(height=500, width=1200, showlegend=False)
  # add subplot of f plotted against the points generated inside the domain to the left of the figure
  fig.add_traces(
      data=[go.Scatter3d(x=X_inner[0], y=X_inner[1], z=f, mode='markers'),],
      rows=1, cols=1
  )
  # add subplot of u_exact plotted against the points generated to check approximations to the middle of the figure
  fig.add_traces(
      data=[go.Scatter3d(x=X_check[0], y=X_check[1], z=u_exact, mode='markers'),],
      rows=1, cols=2
  )
  # add subplot of g plotted against the points generated at the boundary to the right of the figure
  fig.add_traces(
      data=[go.Scatter3d(x=X_boundary[0], y=X_boundary[1], z=g, mode='markers'),],
      rows=1, cols=3
  )
  fig.show()
elif d > 2:
  # create a figure with three subplots (left, middle, right)
  fig = make_subplots(rows=1, cols=3, 
                      subplot_titles=("X_inner, color: f", "X_check, color: u_exact", "X_boundary, color: g"), 
                      specs=[[{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}]])
  fig.update_layout(height=500, width=1200, showlegend=False)
  # add subplot of inner points generated inside the domain colored according to f to the left of the figure
  fig.add_traces(
      data=[go.Scatter3d(x=X_inner[0], y=X_inner[1], z=X_inner[2], 
                        mode='markers', marker= dict(size=5,
                                                      color=f, # set color to an array/list of desired values
                                                      colorscale='Viridis',   # choose a colorscale
                                                      opacity=1), 
                        ),
            ],
      rows=1, cols=1,
  )
  # add subplot of points generated to check approximations colored according to u_exact to the middle of the figure
  fig.add_traces(
      data=[go.Scatter3d(x=X_check[0], y=X_check[1], z=X_check[2], 
                        mode='markers', marker= dict(size=5,
                                                      color=u_exact, # set color to an array/list of desired values
                                                      colorscale='Viridis',   # choose a colorscale
                                                      opacity=1)
                        ),
            ],
      rows=1, cols=2
  )
  # add subplot of points genearted at the boundary colored according to g to the right of the figure
  fig.add_traces(
      data=[go.Scatter3d(x=X_boundary[0], y=X_boundary[1], z=X_boundary[2], 
                        mode='markers', marker= dict(size=5,
                                                      color=g, # set color to an array/list of desired values
                                                      colorscale='Viridis', 
                                                      opacity=1)
                        ),
            ],
      rows=1, cols=3
  )
  fig.show()

# PINN & DRM

The functions in the following cell define how to perform a train step with PINNs and with the DRM. The details of why this results in the neural network approximating the exact solution are explained in **section 2.1 and 2.2** of the thesis.

In [ ]:
def train_step_PINN(X_inner, V_inner, f, X_boundary, V_boundary, g, model, beta, opt):
  # X_inner and X_boundary should contain data of the same dimension
  assert len(X_inner) == len(X_boundary) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_inner[i]) == len(X_inner[i+1]) for i in range(len(X_inner)-1)]) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_boundary[i]) == len(X_boundary[i+1]) for i in range(len(X_boundary)-1)]) 
  # f should contain as many values as we have passed inputs to rhs
  assert len(X_inner[0]) == len(f) 
  # g should contain as many values as we have passed inputs to exact_sol
  assert len(X_boundary[0]) == len(g) 
  
  d = len(X_inner)
  batch_size_inner = len(X_inner[0]) 
  batch_size_boundary = len(X_boundary[0])

  with tf.GradientTape() as tape3:
    with tf.GradientTape(persistent=True) as tape2:
      tape2.watch(X_inner)
      with tf.GradientTape() as tape1:
        tape1.watch(X_inner)
        
        batch = tf.stack(X_inner, axis=1)
        u = model(batch)
        u = tf.reshape(u, [batch_size_inner])
      udx = tape1.gradient(u, X_inner)
    udxx = [tape2.gradient(udx[i], X_inner[i]) for i in range(d)]
    tape2.stop_recording()

    batch_boundary = tf.stack(X_boundary, axis=1)
    u_boundary = model(batch_boundary)
    u_boundary = tf.reshape(u_boundary, [batch_size_boundary])

    laplacian = tf.reduce_sum(udxx, axis=0) 
    loss_inner = V_inner/batch_size_inner * tf.reduce_sum((laplacian+f)**2, axis=0) # -Δu = f 
    loss_boundary = V_boundary/batch_size_boundary * tf.reduce_sum((u_boundary - g)**2, axis=0)
      
      # Alternative balanced loss functions (used in section 3.4)
    # loss_inner = (V_inner/V_boundary) * (1/batch_size_inner) * tf.reduce_sum((laplacian+f)**2, axis=0) # -Δu = f 
    # loss_boundary = (1/batch_size_boundary) * tf.reduce_sum((u_boundary - g)**2, axis=0)
    loss = loss_inner + beta*loss_boundary
  grad = tape3.gradient(loss, model.trainable_weights)
  opt.apply_gradients(zip(grad, model.trainable_weights))

  return loss, loss_inner, loss_boundary

def train_step_DRM(X_inner, V_inner, f, X_boundary, V_boundary, g, model, beta, opt):
  # X_inner and X_boundary should contain data of the same dimension
  assert len(X_inner) == len(X_boundary) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_inner[i]) == len(X_inner[i+1]) for i in range(len(X_inner)-1)]) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_boundary[i]) == len(X_boundary[i+1]) for i in range(len(X_boundary)-1)]) 
  # f should contain as many values as we have passed inputs to rhs
  assert len(X_inner[0]) == len(f) 
  # g should contain as many values as we have passed inputs to exact_sol
  assert len(X_boundary[0]) == len(g) 
  
  batch_size_inner = len(X_inner[0]) 
  batch_size_boundary = len(X_boundary[0])

  with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:
      tape1.watch(X_inner)
      
      batch = tf.stack(X_inner, axis=1)
      u = model(batch)
      u = tf.reshape(u, [batch_size_inner])
    udx = tape1.gradient(u, X_inner)

    batch_boundary = tf.stack(X_boundary, axis=1)
    u_boundary = model(batch_boundary)
    u_boundary = tf.reshape(u_boundary, [batch_size_boundary])
    
                                         #  tf.reduce_sum(tf.pow(udx, 2), axis=0) corresponds to |∇u|²
    loss_inner = V_inner/batch_size_inner * tf.reduce_sum(0.5*tf.reduce_sum(tf.pow(udx, 2), axis=0)-f*u, axis=0) # 1/2*|∇u|²-f*u
    loss_boundary = V_boundary/batch_size_boundary * tf.reduce_sum((u_boundary - g)**2, axis=0)
      
      # Alternative balanced loss functions (used in section 3.4)
    # loss_inner = (V_inner/V_boundary)*(1/batch_size_inner) * tf.reduce_sum(0.5*tf.reduce_sum(tf.pow(udx, 2), axis=0)-f*u, axis=0) # 1/2*|∇u|²-f*u
    # loss_boundary = (1/batch_size_boundary) * tf.reduce_sum((u_boundary - g)**2, axis=0)
    loss = loss_inner + beta*loss_boundary
  grad = tape2.gradient(loss, model.trainable_weights)
  opt.apply_gradients(zip(grad, model.trainable_weights))

  return loss, loss_inner, loss_boundary

## Alt: Train Step Using Finite Differences 

You can execute the following cell in order to train the neural networks with finite differences instead of autodiff. This was for example used for the runtime tests in **section 3.1**.

In order to revert back to using autodiff simply overwrite the functions `train_step_PINN` and `train_step_DRM` by running the last cell again.

In [ ]:
h = 1e-3 # step size of finite differences

def train_step_PINN(X_inner, V_inner, f, X_boundary, V_boundary, g, model, beta, opt):
  # X_inner and X_boundary should contain data of the same dimension
  assert len(X_inner) == len(X_boundary) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_inner[i]) == len(X_inner[i+1]) for i in range(len(X_inner)-1)]) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_boundary[i]) == len(X_boundary[i+1]) for i in range(len(X_boundary)-1)]) 
  # f should contain as many values as we have passed it inputs
  assert len(X_inner[0]) == len(f) 
  # g should contain as many values as we have passed it inputs
  assert len(X_boundary[0]) == len(g) 
  
  d = len(X_inner)
  batch_size_inner = len(X_inner[0]) 
  batch_size_boundary = len(X_boundary[0])
  
  with tf.GradientTape() as tape:
    batch = tf.stack(X_inner, axis=1)

    u = model(batch)
    u = tf.reshape(u, [batch_size_inner])

    laplacian = 0
    for i in range(d):
      u_p = model(batch+tf.one_hot([i], d, h)) # u(x + e_i*h) p: plus
      u_p = tf.reshape(u_p, [batch_size_inner])

      u_m = model(batch-tf.one_hot([i], d, h)) # u(x - e_i*h) m: minus
      u_m = tf.reshape(u_m, [batch_size_inner])

      laplacian += (u_p -2*u +u_m)/h**2

    batch_boundary = tf.stack(X_boundary, axis=1)
    u_boundary = model(batch_boundary)
    u_boundary = tf.reshape(u_boundary, [batch_size_boundary])

    loss_inner = V_inner/batch_size_inner * tf.reduce_sum((laplacian+f)**2, axis=0) # -Δu = f 
    loss_boundary = V_boundary/batch_size_boundary * tf.reduce_sum((u_boundary - g)**2, axis=0)
    loss = loss_inner + beta*loss_boundary
  grad = tape.gradient(loss, model.trainable_weights)
  opt.apply_gradients(zip(grad, model.trainable_weights))

  return loss, loss_inner, loss_boundary

def train_step_DRM(X_inner, V_inner, f, X_boundary, V_boundary, g, model, beta, opt):
  # X_inner and X_boundary should contain data of the same dimension
  assert len(X_inner) == len(X_boundary) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_inner[i]) == len(X_inner[i+1]) for i in range(len(X_inner)-1)]) 
  # Every point in X_inner should have a component in all d coordinates
  assert all([len(X_boundary[i]) == len(X_boundary[i+1]) for i in range(len(X_boundary)-1)]) 
  # f should contain as many values as we have passed it inputs
  assert len(X_inner[0]) == len(f) 
  # g should contain as many values as we have passed it inputs
  assert len(X_boundary[0]) == len(g) 
  
  d = len(X_inner)
  batch_size_inner = len(X_inner[0]) 
  batch_size_boundary = len(X_boundary[0])

  with tf.GradientTape() as tape:
    batch = tf.stack(X_inner, axis=1)

    u = model(batch)
    u = tf.reshape(u, [batch_size_inner])

    nabla_u_2 = 0 # will become ∇u squared component wise
    for i in range(d):
      u_p = model(batch+tf.one_hot([i], d, h)) # u(x + e_i*h) p: plus
      u_p = tf.reshape(u_p, [batch_size_inner])

      nabla_u_2 += ((u_p -u)/h)**2 

    batch_boundary = tf.stack(X_boundary, axis=1)
    u_boundary = model(batch_boundary)
    u_boundary = tf.reshape(u_boundary, [batch_size_boundary])
    
    loss_inner = V_inner/batch_size_inner * tf.reduce_sum(0.5*nabla_u_2-f*u, axis=0) # 1/2*|∇u|²-f*u
    loss_boundary = V_boundary/batch_size_boundary * tf.reduce_sum((u_boundary - g)**2, axis=0)
    loss = loss_inner + beta*loss_boundary
  grad = tape.gradient(loss, model.trainable_weights)
  opt.apply_gradients(zip(grad, model.trainable_weights))

  return loss, loss_inner, loss_boundary

# Tests

## Performance Metrics

Running the following cell, will create the functions used to record the relative $L^2$-, $H^1$-, $L^\infty$- and $C^1$-errors introduced in **section 2.6** of the thesis.

You can define custom metrics by following the structure of the code below.

In [ ]:
# define additional metrics that can be recorded during training

# a metric function for the relative L2 error to wrap in MeanMetricWrapper 
def fn_relative_L2(y_true, y_pred):
  # y_true and y_pred are lists of the form [u, udx[0], ..., udx[d]] 

  # note: because we use the RELATIVE L2 norm the term "Volume/num" in the denominator 
  #       is cancelled by the same term in the enumerator
  return tf.norm(y_true[0]-y_pred[0], 2)/tf.norm(y_true[0], 2)
# create the relative L2 error metric from MeanMetricWrapper
  # Q: why do we need a mean to calc the relative L2 norm?
  # A: we don't. Metrics in tf can record multiple errors before calculating their result
  #    In the case of a MeanMetricWrapper, the result will be the mean of the recorded errors
  #    the metrics used in the training loop will however always record only ONE error 
metric_relative_L2 = keras.metrics.MeanMetricWrapper(fn_relative_L2, name='relative_L2_error')


# a metric function for the relative H1 error to wrap in MeanMetricWrapper 
def fn_relative_H1(y_true, y_pred):
  # y_true and y_pred are lists of the form [u, udx[0], ..., udx[d-1]] 
  if len(y_true) <= 1: return np.nan
  y_difference, y = 0, 0
  for i in range(len(y_true)):
    y_difference += tf.norm(y_true[i]-y_pred[i], 2)
    y += tf.norm(y_true[i], 2)
  # note: because we use the RELATIVE H1 norm the term "Volume/num" in the denominator 
  #       is cancelled by the same term in the enumerator and an increased number of 
  #       partial derivatives in the enumarator is balanced by the same increase in the denominator
  return y_difference/y
# create the relative H1 error metric from MeanMetricWrapper
metric_relative_H1 = keras.metrics.MeanMetricWrapper(fn_relative_H1, name='relative_H1_error')


# a metric function for the relative L_oo error to wrap in MeanMetricWrapper 
def fn_relative_L_oo(y_true, y_pred):
  # y_true and y_pred are lists of the form [u, udx[0], ..., udx[d-1]] 
  return tf.norm(y_true[0]-y_pred[0], np.inf)/tf.norm(y_true[0], np.inf)
# create the relative L_oo error metric from MeanMetricWrapper
metric_relative_L_oo = keras.metrics.MeanMetricWrapper(fn_relative_L_oo, name='relative_L_oo_error')

# a metric function for the relative C1 error to wrap in MeanMetricWrapper 
def fn_relative_C1(y_true, y_pred):
  # y_true and y_pred are lists of the form [u, udx[0], ..., udx[d-1]] 
  if len(y_true) <= 1: return np.nan
  y_difference, y = 0, 0
  for i in range(len(y_true)):
    y_difference += tf.norm(y_true[i]-y_pred[i], np.inf)
    y += tf.norm(y_true[i], np.inf)
  return y_difference/y
# create the relative c1 error metric from MeanMetricWrapper
metric_relative_C1 = keras.metrics.MeanMetricWrapper(fn_relative_C1, name='relative_C1_error')

## The "Test API"

In order to allow the simple creation of a large number of different tests we wrote a small "Test API".

> A test can be easily constructed by creating an instance of the Test class and passing all parameters we want to change. The unspecified parameters are asigned default values, which are defined in the `__init__` method.

We typically wanted to create multiple tests at once, 
where all tests had a range of parameters in common and only a few parameters changed from test to test. For this we wrote the function `create_tests(*tests, group='', **params)`, which takes a list of Tests and parameters as its input and inserts the list of parameters into every Test. 

A Test created in this way is also assigned a *group* defined by the parameter `group` that is passed to `create_tests`. A Test can also have a name that is displayed during training and is used for logging. 

> Group names and test names can be generated automatically by the parameters of the group/test. For this simply write an underscore after the name of all the parameters you want to include in the name. For example: 
```
create_tests(Test(learning_rate_ = 10, beta = 0), epochs_ = 2, logging_interval = 1) 
```
will have the name 'learning_rate=10' and be in the group 'epochs=2'.

In [ ]:
# this is mearly a parser in order to convert attributes of a class into an easier to read format
# this is for example used in order to create the names of test-groups or test-names
# but also for the description of a test
def to_readable_str(val):
  string = ''
  place_holder = '{}'
  iterator = [val]
  try: 
    if type(val) == str:
      place_holder = '"{}"'
    else:
      iterator = iter(val)
      place_holder = '[{}]'
  finally:
    for el in iterator:
      string += ', ' if string != '' else ''
      if 'name' in dir(el): 
        string += el.name
      elif '__name__' in dir(el):
        string += el.__name__
      else:
        string += str(el)
    return place_holder.format(string)
    
# the Test class with default test values
class Test:
  # change parameters here to change them for every test at once
  def __init__(self, *, name = '',
               logging_interval = 100, printing_interval = 1000, # both in epochs
               input_dim = 3, num_blocks = 4, nodes_per_layer = 10, 
               activation = keras.activations.swish,  
                # you can use any function defined in keras.activations
                # or define your own using keras.layers.Activation(lambda x: x, name='example activation')
               generate_interior = generate_interior_via_LHS, generate_interior_check = generate_interior_randomly, 
               generate_boundary = generate_boundary_via_LHS, generate_boundary_check = generate_boundary_randomly, 
               rhs = None, exact_sol = None, # TODO: do you want to specify not None default values for this? 
               domain_params = [-1.0, 1.0], # [a: interval start, b: interval end] the hypercube is [a,b]^input_dim
               interior_params = [1_000], interior_check_params = [10_000], # [batch_size_inner: size of batches containing points in the interior of the domain]
               boundary_params = [100], boundary_check_params = [1000], # num_boundary_along_one_axis: number of boundary points along one axis
               epochs = 10_000, beta = 100, learning_rate = 0.001, 
               optimizer = keras.optimizers.Adam, metrics = [metric_relative_L2, metric_relative_H1, metric_relative_L_oo], **kwargs):
    self.name = name # tests have names in order to more easily identify them (e.g. test names are part of the file names of logs)
    self.group = '' # tests belong to groups in order to better organize them (e.g. tests in one group will create logs in a seperate folder)

    self.logging_interval = logging_interval # interval in epochs between logging 
    self.printing_interval = printing_interval # inteval in epochs between printing

    self.input_dim = input_dim # input dimension of neural network
    self.num_blocks = num_blocks # number of residual blocks
    self.nodes_per_layer = nodes_per_layer # the layer size used for all hidden layers 
    self.activation = activation # the activation function used for all hidden layers

    self.generate_interior = generate_interior # TODO: explain
    self.generate_interior_check = generate_interior_check # TODO: explain
    self.generate_boundary = generate_boundary # TODO: explain
    self.generate_boundary_check = generate_boundary_check # TODO: explain

    self.rhs = rhs # TODO: explain
    self.exact_sol = exact_sol # TODO: explain

    self.domain_params = domain_params # a tuple defining the parameters regarding the domain needed by the generators 
    self.interior_params = interior_params # a tuple defining the parameters (appart from domain params) generate_interior expects
    self.interior_check_params = interior_check_params # a tuple defining the parameters (appart from domain params) generate_interior_check expects
    self.boundary_params = boundary_params # a tuple defining the parameters (appart from domain params) generate_boundary expects
    self.boundary_check_params = boundary_check_params # a tuple defining the parameters (appart from domain params) generate_boundary_check expects
    # those parameters are passed like this: generate_abc(*domain_params, *abc_params)

    self.epochs = epochs # (in this case) total number of training steps
    self.beta = beta # weight of boundary condition penalty term
    self.learning_rate = learning_rate # learning rate passed to the optimization algorithm used
    self.optimizer = optimizer # optimization algorithm used to update weights and biases
    self.metrics = metrics # list of metrics that should be logged and printed during each test

    self.kwargs = kwargs

  def description(self):
    # this method returns a complete description of the test 
    text = ''
    for attr in dir(self):
      if not attr.startswith('__') and attr not in ['kwargs', 'description']:
        text += ', ' if text != '' else ''
        text += f"{attr}={to_readable_str(getattr(self, attr))}"
    return text

# use this method to create multiple tests at once that share some parameters
  # create_tests automatically creates a group if none was passed (i.e. group='') based on the parameters passed to params that end with _
  # create_tests automatically creates names for tests without names (i.e. name = '') based on the parameters in kwargs of the test that end with _
  # e.g.
  # the test in the list created by create_tests(Test(learning_rate_ = 10, beta = 0), epochs_ = 2, logging_interval = 1) 
  # will have the name 'learning_rate=10' and be in the group 'epochs=2' and 
  # (beta and logging_interval are not included because they don't end in an underscore "_")
def create_tests(*tests, group='', **params):
  for t in tests:
    t.group = group

    # go over kwargs
    for key in t.kwargs:
      attr = key[:-1]
      if key.endswith('_') and hasattr(t, attr):
        val = t.kwargs[key]
        # group params always override test params
        if key in params: val = params[key]
        if attr in params: val = params[attr]
        setattr(t, attr, val)
        t.name += ', ' if t.name != '' else ''
        t.name += f"{attr}={to_readable_str(val)}"
    # go over params
    for key in params:
      attr = key[:-1] if key.endswith('_') else key
      if hasattr(t, attr):
        setattr(t, attr, params[key])
        # if no group, set group
        if key.endswith('_'):
          t.group += ', ' if t.group != '' else ''
          t.group += f"{attr}={to_readable_str(params[key])}"
  return tests

## Define All the Tests to Be Run

In [ ]:
# all tests in the tests variable will be executed in the training loop
tests = []

# # # # # # # # # # # # # # #
# define all the tests here #
# # # # # # # # # # # # # # #

# for example
activation_relu_x3 = layers.Activation(lambda x: tf.pow(tf.maximum(0.0,(x/5)), 3), name='relu_x_div_5_pow_3')
tests += create_tests(Test(domain_params_=[1.0, 0.5*math.pi]), Test(domain_params_=[1.0, 1.0*math.pi]),
                      Test(domain_params_=[1.0, 1.5*math.pi]), Test(domain_params_=[1.0, 2.0*math.pi]),
                      input_dim=2, epochs=5_000, activation_=activation_relu_x3, 
                      interior_params=[1_000], interior_check_params=[10_000], 
                      generate_interior=generate_interior_pie_randomly, generate_interior_check=generate_interior_pie_randomly,
                      boundary_params=[10, 100, 10], boundary_check_params=[100, 1000, 100], 
                      generate_boundary=generate_boundary_pie_randomly, generate_boundary_check=generate_boundary_pie_randomly, 
                      rhs=rhs_zero, exact_sol=sol_r_sin_phi)

# generate a summary of the all defined tests 
last_group = None
for index, test in enumerate(tests, 1):
  if last_group != test.group:
    print(80*'-')
    print(f"Group: {test.group}")
    last_group = test.group
  print(f"  Test #{index}:\t{test.name}")
print(80*'=')
print(f"Total number of tests: {len(tests)}")

# Training loop

In the beginning of **section 2** of the thesis, we sketched out a training loop in pseudocode. In this section, the real counter part can be found. 

Executing the cell containing the training loop will commence
training using PINNs and the DRM for all previously defined tests. 

After every `printing_interval` epochs, the current losses and all values of all performance metrics will be displayed.
After every `logging_interval` epochs they will be logged.
(`printing_interval` and `logging_interval` are parameters that can be passed to a Test)

The logging structure is the following:
```
{log_base_route}/group#1/name#1
```
If the group already exists, the number behind group will increase to the next largest number that is not yet used. The same holds for the test name.
The `log_base_rout` is either `logs` if the logs are not saved permanently in google drive or `content/drive/MyDrive/PINN_vs_DRM_tests/logs` otherwise.
Both paths are set automatically.

In [ ]:
# this function calculates the values of the exact solution and 
# PINNs' and DRM's approximation of it, as well as all corresponding derivatives 
# at a given set of points (X_check).
# those can then be used to calculate (relative) errors
@tf.function 
def calc_u_check(X_check, exact_sol, model_PINN, model_DRM):
  batch_size_check = len(X_check[0])
  with tf.GradientTape(persistent=True) as tape:
    tape.watch(X_check)
    
    u_exact = exact_sol(X_check)
    u_exact = tf.reshape(u_exact, [batch_size_check])

    batch = tf.stack(X_check, axis=1)
    u_PINN = model_PINN(batch)
    u_PINN = tf.reshape(u_PINN, [batch_size_check])
    u_DRM = model_DRM(batch)
    u_DRM = tf.reshape(u_DRM, [batch_size_check])
  udx_exact = tape.gradient(u_exact, X_check)

  udx_PINN = tape.gradient(u_PINN, X_check)
  udx_DRM = tape.gradient(u_DRM, X_check)
  tape.stop_recording()
  
  return u_exact, udx_exact, u_PINN, udx_PINN, u_DRM, udx_DRM

# this function calculates the same variables as the one above but in this case for boundary points
# we therefore do not need to calculate the derivatives
def calc_u_boundary_check(X_boundary_check, exact_sol, model_PINN, model_DRM):
  u_boundary_exact = exact_sol(X_boundary_check)
  batch = tf.stack(X_boundary_check, axis=1)
  u_boundary_PINN = model_PINN(batch)
  u_boundary_DRM = model_DRM(batch)
  return u_boundary_exact, u_boundary_PINN, u_boundary_DRM

❗ **Attention:** Running this cell will start the training loop!,
which will immediately create the first logging folder and may take a while till completion. 

A running cell can be stopped by clicking on the corresponding button at the top left of the cell.

In [ ]:
Path(log_base_route).mkdir(parents=True, exist_ok=True)

last_group = None
for index, test in enumerate(tests, 1):
  print(80*"#")
  print(f"working on {index}/{len(tests)} {test.group}/{test.name} ")
  print(80*"#")

  # # # # # # # # # # # # # # # #
  # set up logging and printing #
  # # # # # # # # # # # # # # # #
  
  # create directory for logs of the current test
  #   add test group and name to route to structure tests
  #     if group name already exists append #num appropriately
  #     if test name already exists append #num appropriately
  if last_group != test.group:
    test_num_dict = defaultdict(lambda:0)
    group_num = 0
    for group_name in listdir(log_base_route):
      if re.match(f'(^{test.group}$)|(^{test.group}#[0-9]*$)', group_name):
        match = re.search('#[0-9]*$', group_name)
        group_num = max(group_num, int(match.group(0)[1:]) if match != None else 1)
    group_dir = f'{log_base_route}/{test.group}#{group_num+1}'
    last_group = test.group
  test_num_dict[test.name] += 1
  log_dir = f'{group_dir}/{test.name}#{test_num_dict[test.name]}'
  # create complementary file writer
  summary_writer = tf.summary.create_file_writer(log_dir)
  summary_writer.set_as_default()

  tf.summary.text('test description', test.description(), 0)
  start_time = datetime.utcnow()
  start_time_CPU = time.process_time()
  tf.summary.text('start time (in UTC)', str(start_time), 0)
  tf.summary.flush()

  # # # # # # # # #
  # create  model #
  # # # # # # # # #

  # building the models
  models = []
  input_dim = test.input_dim
  for _ in range(2): # build one model for PINN and one for the DRM
    inputs = keras.Input(shape=(input_dim,), batch_size=None) # None means batch size is determined dynamically
    x = layers.Dense(test.nodes_per_layer, activation=None if test.num_blocks > 0 else test.activation)(inputs) 
    #   in order to add the input via the skip connection to the output of the first res block, 
    #   the dimensions have to match. This can either be achieved via the linear transformation above
    #   or (when dim input < dim res block) by padding zeros to the input
    # x = tf.pad(inputs, [[0,0], [0,test.nodes_per_layer-test.input_dim]]) # pad the input with zeros until the new input dimension matches nodes_per_layer
    #                                                                      # the first padding [0,0] is for the batch "dimension"
    for k in range(test.num_blocks):
      tmp = layers.Dense(test.nodes_per_layer, activation=test.activation)(x)
      tmp = layers.Dense(test.nodes_per_layer, activation=test.activation)(tmp)
      x = layers.add([tmp, x]) # this is the skip/residual connection
    outputs = layers.Dense(1)(x)
    models += [keras.Model(inputs=inputs, outputs=outputs)]
  model_PINN = models[0]
  model_DRM = models[1]
    # in order to guarantee the same initial conditions, 
    # copy the weights from the PINN model to the DRM model
  model_DRM.set_weights(model_PINN.get_weights()) 

  # print model summary
  # model_PINN.summary()
  
  # # # # # # # # # # # #
  # training loop setup #
  # # # # # # # # # # # #

  opt = test.optimizer(test.learning_rate)
  domain_params = test.domain_params

  # generators
  generate_interior = lambda: test.generate_interior(input_dim, *domain_params, *test.interior_params)
  generate_interior_check = lambda: test.generate_interior_check(input_dim, *domain_params, *test.interior_check_params)
  generate_boundary = lambda: test.generate_boundary(input_dim, *domain_params, *test.boundary_params)
  generate_boundary_check = lambda: test.generate_boundary_check(input_dim, *domain_params, *test.boundary_check_params)

  # rhs and exact solution
  rhs = lambda X: test.rhs(*domain_params, X)
  exact_sol = lambda X: test.exact_sol(*domain_params, X)

  # train steps
  train_step_PINN_acc = tf.function(train_step_PINN) # accelerate train step with tf.function
  train_step_DRM_acc = tf.function(train_step_DRM) # accelerate train step with tf.function

  # # # # # # # # # # # # # # # # # #
  # train model using  PINN and DRM #
  # # # # # # # # # # # # # # # # # #

  for epoch in range(test.epochs+1): # use range(epochs PLUS ONE) in order to finish at epoch == epochs
    if epoch == 1: # save the "setup" time (time until one epoch is finished) this includes in particular the tracing time
      tf.summary.text('setup duration', f'Wall time: {datetime.utcnow() - start_time}', 0)
      tf.summary.text('setup duration', f'CPU time: {time.process_time() - start_time_CPU}', 1)
      tf.summary.flush()

    # determine if to log or print at this epoch
    logging_on = epoch % test.logging_interval == 0 or epoch == test.epochs
    printing_on = epoch % test.printing_interval == 0 or epoch == test.epochs
    
    # Q: why do we calc u_PINN and u_DRM before we perform the train step?
    # A: the loss returned by train_step() is always the loss BEFORE the train step
    #    in order for the metrics and loss to be in sync, we thus need to compute u_PINN and u_DRM before the train step
    if logging_on or printing_on:
      # calculate exact and approx solution and their derivatives in the interior needed for logging or printing
      X_check, _ = generate_interior_check()
      u_exact, udx_exact, u_PINN, udx_PINN, u_DRM, udx_DRM = calc_u_check(X_check, exact_sol, model_PINN, model_DRM)
      # calculate exact and approx solution at the boundary needed for logging or printing
      X_boundary_check, _ = generate_boundary_check()
      u_boundary_exact, u_boundary_PINN, u_boundary_DRM = calc_u_boundary_check(X_boundary_check, exact_sol, model_PINN, model_DRM)

    # execute one train step for PINN and the DRM
    X_inner, V_inner = generate_interior()
    f = rhs(X_inner)
    X_boundary, V_boundary = generate_boundary()
    g = exact_sol(X_boundary)
    loss_PINN, loss_inner_PINN, loss_boundary_PINN = train_step_PINN_acc(X_inner, V_inner, f, X_boundary, V_boundary, g, model_PINN, test.beta, opt)
    loss_DRM, loss_inner_DRM, loss_boundary_DRM = train_step_DRM_acc(X_inner, V_inner, f, X_boundary, V_boundary, g, model_DRM, test.beta, opt)

    # printing to the console and logging
    if logging_on or printing_on:
      # log and print losses at the specified epoch
      if logging_on:
          tf.summary.scalar('PINN loss', loss_PINN, step=epoch)
          tf.summary.scalar('PINN loss_inner', loss_inner_PINN, step=epoch)
          tf.summary.scalar('PINN loss_boundary', loss_boundary_PINN, step=epoch)
          tf.summary.scalar('DRM loss', loss_DRM, step=epoch)
          tf.summary.scalar('DRM loss_inner', loss_inner_DRM, step=epoch)
          tf.summary.scalar('DRM loss_boundary', loss_boundary_DRM, step=epoch)
      if printing_on:
        print(f"loss at epoch {epoch}/{test.epochs}={math.floor(epoch/test.epochs*100)}%:\tPINN {loss_PINN}\tDRM {loss_DRM}")

      # loop over all metrics of interest to this test
      for metric in test.metrics:
        # calc metric for PINN
          # inner error
        metric.update_state(tf.stack([u_exact, *udx_exact]), tf.stack([u_PINN, *udx_PINN]))
        metric_res_PINN = float(metric.result())
        metric.reset_states()
          # boundary error
        metric.update_state(tf.stack([u_boundary_exact]), tf.stack([u_boundary_PINN]))
        metric_res_boundary_PINN = float(metric.result())
        metric.reset_states()

        # calc metric for the DRM
          # inner error
        metric.update_state(tf.stack([u_exact, *udx_exact]), tf.stack([u_DRM, *udx_DRM]))
        metric_res_DRM = float(metric.result())
        metric.reset_states()
          # boundary error
        metric.update_state(tf.stack([u_boundary_exact]), tf.stack([u_boundary_DRM]))
        metric_res_boundary_DRM = float(metric.result())
        metric.reset_states()

        # log and print metrics at the specified epoch
        if logging_on:
          tf.summary.scalar('PINN ' + metric.name, metric_res_PINN, step=epoch)
          tf.summary.scalar('PINN ' + metric.name + ' boundary', metric_res_boundary_PINN, step=epoch)
          tf.summary.scalar('DRM ' + metric.name, metric_res_DRM, step=epoch)
          tf.summary.scalar('DRM ' + metric.name + ' boundary', metric_res_boundary_DRM, step=epoch)
        if printing_on:
          print(f"{metric.name} at epoch {epoch}/{test.epochs}={math.floor(epoch/test.epochs*100)}%:\tPINN {metric_res_PINN}\tDRM {metric_res_DRM}") 
      tf.summary.flush()
  tf.summary.text('test duration', f'Wall time: {datetime.utcnow() - start_time}', 0)
  tf.summary.text('test duration', f'CPU time: {time.process_time() - start_time_CPU} seconds', 1)
  tf.summary.flush()
  print(f'Wall time: {datetime.utcnow() - start_time}')
  print(f'CPU time: {time.process_time() - start_time_CPU} seconds')

# Results

After training you can immediatly inspect your training data by 
opening it in the TensorBoard. The following cell will do that for all the tests found anywhere inside `log_base_route`. 

If the number of tests in this folder becomes to large you can specify a concrete path for example with 
`%tensorboard --logdir "content/drive/MyDrive/PINN_vs_DRM_tests/logs/group/name"` 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_base_route}